In [1]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (18.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from pykrx import stock
sns.set(font='NaNumBarunGothic')


import warnings
warnings.filterwarnings('ignore')
import re
pd.options.display.max_columns = None

In [17]:
# 기준이 되는 재무제표 데이터 끌어오기(여기에 있는 종목만 선정하기 위해)
fs = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/real_real_df.csv')

# 종목코드 6자리로 채워주기
fs['Stock'] = fs['Stock'].astype('str')
fs['Stock'] = fs['Stock'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기

stock_code = list(fs['Stock'])

In [18]:
# Piotroski F - score

piotroski_data = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/Piotroski_data.csv')

piotroski_data.head(10)
def make_preprocessing(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4].split('/')[0]})")

  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df = df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code가 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace=True)
  df = df.reset_index(drop = True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)



  # 문자열 float로 변경
  for i in df.columns[3:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '', x))
    df[i] = df[i].astype('float')


  # null인 경우 0으로 채운다. -> 이거 잠깐 보류
  # 총자본회전율(2017) - 1개
  for i in df.columns:
    if '총자본회전율' in i:
      df[i].fillna(0, inplace=True)
      


  return df


piotroski = make_preprocessing(piotroski_data)
piotroski

,KIS,Stock,Name,총자본회전율 (2017),총자본영업이익율 (2017),매출총이익(손실) (2017),당기순이익(손실) (2017),유동자산(계) (2017),유동부채(계) (2017),비유동부채(계) (2017),영업활동으로인한현금흐름 (2017),총자본회전율 (2018),총자본영업이익율 (2018),매출총이익(손실) (2018),당기순이익(손실) (2018),유동자산(계) (2018),유동부채(계) (2018),비유동부채(계) (2018),영업활동으로인한현금흐름 (2018),총자본회전율 (2019),총자본영업이익율 (2019),매출총이익(손실) (2019),당기순이익(손실) (2019),유동자산(계) (2019),유동부채(계) (2019),비유동부채(계) (2019),영업활동으로인한현금흐름 (2019),총자본회전율 (2020),총자본영업이익율 (2020),매출총이익(손실) (2020),당기순이익(손실) (2020),유동자산(계) (2020),유동부채(계) (2020),비유동부채(계) (2020),영업활동으로인한현금흐름 (2020),총자본회전율 (2021),총자본영업이익율 (2021),매출총이익(손실) (2021),당기순이익(손실) (2021),유동자산(계) (2021),유동부채(계) (2021),비유동부채(계) (2021),영업활동으로인한현금흐름 (2021)
0,036644,389500,(주)에스비비테크,0.53,4.37,2.248718e+09,2.678790e+08,7.090299e+09,7.257253e+09,1.411578e+09,7.497610e+08,0.44,-5.23,1.620293e+09,-1.917031e+09,1.356880e+10,8.723921e+09,1.099427e+10,6.075800e+07,0.27,-17.06,1.152950e+08,-4.789125e+09,8.293176e+09,3.591423e+09,1.568548e+10,-2.686112e+09,0.38,-15.18,1.080739e+09,-3.276516e+09,4.417464e+09,7.995986e+09,5.215853e+09,-1.165070e+09,0.42,-13.65,1.165755e+09,-2.815924e+09,7.302344e+09,6.321137e+09,4.714673e+09,-1.715607e+09
1,PM6283,441270,(주)파인엠텍,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,497730,296640,(주)이노룰스,1.20,18.12,6.174259e+09,7.684080e+08,4.828215e+09,5.665559e+09,1.045236e+09,NaN,0.75,3.88,5.928031e+09,1.770280e+08,6.745808e+09,7.579176e+09,1.856800e+09,1.148444e+09,0.84,17.43,7.203013e+09,6.186920e+08,7.552133e+09,4.520995e+09,2.587218e+09,4.188603e+09,0.74,8.25,6.224520e+09,1.591312e+09,7.703371e+09,5.275242e+09,6.478850e+08,1.613872e+09,1.05,21.71,1.000607e+10,3.009484e+09,1.128946e+10,3.720986e+09,7.612350e+08,2.473444e+09
3,499459,417970,모델솔루션(주),1.49,15.95,1.145713e+10,3.048896e+09,8.925932e+09,6.779529e+09,1.290421e+10,3.615779e+09,1.49,25.30,1.561273e+10,7.210181e+09,1.865476e+10,8.760334e+09,1.269288e+10,5.469434e+09,1.40,21.58,1.661634e+10,7.270528e+09,2.073613e+10,1.861937e+10,2.074573e+09,1.075707e+10,1.26,15.68,1.370890e+10,1.299209e+09,2.207469e+10,1.759698e+10,1.989822e+09,8.304769e+09,1.33,18.82,1.607404e+10,7.086006e+09,2.716175e+10,9.062293e+09,1.216936e+10,9.397543e+09
4,HV2035,393890,더블유씨피(주),0.00,-1.75,NaN,-1.886107e+09,1.413538e+10,4.385492e+09,1.002453e+09,-3.320733e+09,0.00,-8.72,NaN,-9.513513e+09,7.660966e+09,6.746037e+09,3.823229e+09,-8.218473e+09,0.12,-1.72,-6.019870e+08,-6.513745e+09,1.528209e+11,4.331020e+10,1.934810e+11,-2.835368e+10,0.23,2.05,1.552471e+10,-3.117833e+10,1.238286e+11,1.279343e+11,2.318694e+11,-2.842428e+10,0.32,7.09,5.027255e+10,-9.963987e+09,2.144150e+11,1.283054e+11,9.595434e+10,8.574900e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1440,I73639,950110,SBI핀테크솔루션즈(주),0.70,12.53,1.511315e+10,2.833042e+09,1.303828e+10,6.999535e+09,4.228802e+09,4.281560e+09,0.45,8.05,1.663058e+10,3.963095e+09,1.317464e+10,2.285635e+10,1.800394e+10,2.682301e+09,0.28,2.30,1.386710e+10,-1.268170e+08,1.512503e+10,2.688037e+10,1.298102e+10,3.725843e+09,0.29,8.60,2.274169e+10,7.710943e+09,5.957791e+10,5.033506e+10,3.870147e+10,3.268879e+09,0.16,0.92,1.853943e+10,7.052749e+09,1.420810e+11,1.156883e+11,7.796255e+10,-8.972330e+10
1441,125028,046970,(주)우리로,0.31,-9.01,1.414464e+09,-1.053593e+10,1.895801e+10,1.748583e+10,1.355701e+09,3.767429e+09,0.52,4.43,5.406685e+09,-3.391840e+08,2.964992e+10,3.636824e+10,1.011553e+09,2.333764e+09,1.41,0.32,5.001209e+09,-8.603656e+09,2.888815e+10,4.147360e+10,2.207496e+09,1.597561e+09,1.50,-0.39,5.079578e+09,-1.162556e+10,2.448862e+10,3.648397e+10,2.193582e+09,9.402770e+08,1.64,5.31,8.417236e+09,1.335246e+10,4.079365e+10,1.814330e+10,2.815891e+09,-2.713149e+09
1442,383046,113810,디젠스(주),0.61,-5.10,2.359626e+09,-7.515252e+09,2.590133e+10,4.189249e+10,1.218382e+10,1.984220e+09,0.50,-3.98,2.939083e+09,-1.185408e+1

In [19]:
piotroski.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1445 entries, 0 to 1444
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   KIS                  1445 non-null   object 
 1   Stock                1445 non-null   object 
 2   Name                 1445 non-null   object 
 3   총자본회전율 (2017)        1445 non-null   float64
 4   총자본영업이익율 (2017)      1388 non-null   float64
 5   매출총이익(손실) (2017)     1347 non-null   float64
 6   당기순이익(손실) (2017)     1388 non-null   float64
 7   유동자산(계) (2017)       1380 non-null   float64
 8   유동부채(계) (2017)       1380 non-null   float64
 9   비유동부채(계) (2017)      1319 non-null   float64
 10  영업활동으로인한현금흐름 (2017)  1359 non-null   float64
 11  총자본회전율 (2018)        1445 non-null   float64
 12  총자본영업이익율 (2018)      1400 non-null   float64
 13  매출총이익(손실) (2018)     1362 non-null   float64
 14  당기순이익(손실) (2018)     1401 non-null   float64
 15  유동자산(계) (2018)       1393 non-null   f

In [20]:
# fs 데이터에 있는 종목코드만 대상으로 진행하기
tmp = []
piotroski_df = pd.DataFrame()
for i in piotroski['Stock'].unique():
  if i in fs['Stock'].unique():
    idx = list(piotroski[piotroski['Stock']==i].index.values).pop()
    tmp.append(idx)
piotroski_df  =pd.DataFrame(piotroski, index = tmp).reset_index(drop=True)

In [21]:
piotroski_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   KIS                  1279 non-null   object 
 1   Stock                1279 non-null   object 
 2   Name                 1279 non-null   object 
 3   총자본회전율 (2017)        1279 non-null   float64
 4   총자본영업이익율 (2017)      1279 non-null   float64
 5   매출총이익(손실) (2017)     1279 non-null   float64
 6   당기순이익(손실) (2017)     1279 non-null   float64
 7   유동자산(계) (2017)       1279 non-null   float64
 8   유동부채(계) (2017)       1279 non-null   float64
 9   비유동부채(계) (2017)      1230 non-null   float64
 10  영업활동으로인한현금흐름 (2017)  1266 non-null   float64
 11  총자본회전율 (2018)        1279 non-null   float64
 12  총자본영업이익율 (2018)      1279 non-null   float64
 13  매출총이익(손실) (2018)     1279 non-null   float64
 14  당기순이익(손실) (2018)     1279 non-null   float64
 15  유동자산(계) (2018)       1279 non-null   f

In [25]:
piotroski_df[piotroski_df['영업활동으로인한현금흐름 (2017)'].isna()]['Stock'].values

array(['296640', '402030', '399720', '288980', '251120', '372800',
       '382840', '367000', '314930', '356890', '352770', '337930',
       '347770'], dtype=object)

In [14]:
piotroski_df.columns

Index(['KIS', 'Stock', 'Name', '총자본회전율 (2017)', '총자본영업이익율 (2017)',
       '매출총이익(손실) (2017)', '당기순이익(손실) (2017)', '유동자산(계) (2017)',
       '유동부채(계) (2017)', '비유동부채(계) (2017)', '영업활동으로인한현금흐름 (2017)',
       '총자본회전율 (2018)', '총자본영업이익율 (2018)', '매출총이익(손실) (2018)',
       '당기순이익(손실) (2018)', '유동자산(계) (2018)', '유동부채(계) (2018)',
       '비유동부채(계) (2018)', '영업활동으로인한현금흐름 (2018)', '총자본회전율 (2019)',
       '총자본영업이익율 (2019)', '매출총이익(손실) (2019)', '당기순이익(손실) (2019)',
       '유동자산(계) (2019)', '유동부채(계) (2019)', '비유동부채(계) (2019)',
       '영업활동으로인한현금흐름 (2019)', '총자본회전율 (2020)', '총자본영업이익율 (2020)',
       '매출총이익(손실) (2020)', '당기순이익(손실) (2020)', '유동자산(계) (2020)',
       '유동부채(계) (2020)', '비유동부채(계) (2020)', '영업활동으로인한현금흐름 (2020)',
       '총자본회전율 (2021)', '총자본영업이익율 (2021)', '매출총이익(손실) (2021)',
       '당기순이익(손실) (2021)', '유동자산(계) (2021)', '유동부채(계) (2021)',
       '비유동부채(계) (2021)', '영업활동으로인한현금흐름 (2021)'],
      dtype='object')

# 결측치 처리
비유동부채 = 부채총계 - 유동부채 이므로 부채총계 데이터를 다시 갖오와서 merge 적용.

In [47]:
# 연도별 부채총계 데이터 받아오기
liability_data = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/Piotroski_data2.csv')
liability_data.head(10)

,Massive Download,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Report as of : 2022-10-31,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Portfolio - kosdaq,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,KRW,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,2017/Annual,2018/Annual,2019/Annual,2020/Annual,2021/Annual
5,KIS,Stock,Name,부채총계/018000,부채총계/018000,부채총계/018000,부채총계/018000,부채총계/018000
6,036644,389500,(주)에스비비테크,"8,668,831,000.","19,718,189,000.","19,276,907,000.","13,211,839,000.","11,035,810,000."
7,PM6283,441270,(주)파인엠텍,NaN,NaN,NaN,NaN,NaN
8,497730,296640,(주)이노룰스,"6,710,795,000.","9,435,975,000.","7,108,213,000.","5,923,127,000.","4,482,222,000."
9,499459,417970,모델솔루션(주),"19,683,736,000.","21,453,211,000.","20,693,945,000.","19,586,807,000.","21,231,656,000."


In [51]:
liability_data = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/Piotroski_data2.csv')

def make_preprocessing_liability(df):
  col_name = ['KIS', 'Stock', 'Name']
  for i in df.columns[3:]:
    col_name.append(f"{df[i].iloc[5].split('/')[0]} ({df[i].iloc[4].split('/')[0]})")

  df.columns = col_name
  df.drop([0,1,2,3,4,5], axis=0, inplace=True)
  df = df.reset_index(drop=True)

  # 뒤에 널값인 데이터도 같이 들어왔는데 같이 삭제
  last_index = df[df['Name']=='(주)아바텍'].index # 데이터가 들어있는 맨 마지막 인덱스. 이 이후꺼를 삭제시키면 된다.
  df = df.iloc[:list(df[df['Name']=='(주)아바텍'].index).pop()+1, :]

  # stock code가 없는 기업 삭제
  del_index = list(df[df['Stock'].isnull()].index)
  df.drop(del_index, axis=0, inplace=True)
  df = df.reset_index(drop = True)

  # 인수합병된 기업들 빼기
  df['인수합병여부'] = 0
  df['인수합병여부'] = df.apply(lambda x:1 if '인수목적' in x['Name'] else 0,axis=1)

  df = df[df['인수합병여부']==0]
  df.drop('인수합병여부', axis=1, inplace =True)
  df = df.reset_index(drop=True)



  # 문자열 float로 변경
  for i in df.columns[3:]:
    df[i] = df[i].astype('str')
    df[i] = df[i].apply(lambda x: re.sub(',', '', x))
    df[i] = df[i].astype('float')

  return df

liability_data = make_preprocessing_liability(liability_data)
liability_data


tmp = []
liability_df = pd.DataFrame()
for i in liability_data['Stock'].unique():
  if i in fs['Stock'].unique():
    idx = list(liability_data[liability_data['Stock']==i].index.values).pop()
    tmp.append(idx)
liability_df  =pd.DataFrame(liability_data, index = tmp).reset_index(drop=True)

liability_df

,KIS,Stock,Name,부채총계 (2017),부채총계 (2018),부채총계 (2019),부채총계 (2020),부채총계 (2021)
0,036644,389500,(주)에스비비테크,8.668831e+09,1.971819e+10,1.927691e+10,1.321184e+10,1.103581e+10
1,497730,296640,(주)이노룰스,6.710795e+09,9.435975e+09,7.108213e+09,5.923127e+09,4.482222e+09
2,499459,417970,모델솔루션(주),1.968374e+10,2.145321e+10,2.069394e+10,1.958681e+10,2.123166e+10
3,HS8995,314140,(주)알피바이오,2.455539e+10,3.689531e+10,5.919584e+10,6.941174e+10,7.881001e+10
4,867055,129920,(주)대성하이텍,7.701378e+10,7.676503e+10,7.503042e+10,8.670698e+10,1.117834e+11
...,...,...,...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),1.122834e+10,4.086029e+10,3.986139e+10,8.903653e+10,1.936508e+11
1275,125028,046970,(주)우리로,1.884153e+10,3.737979e+10,4.368110e+10,3.867756e+10,2.095919e+10
1276,383046,113810,디젠스(주),5.407632e+10,4.229159e+10,4.154523e+10,4.215186e+10,4.218763e+10
1277,F11411,155650,와이엠씨(주),1.452742e+11,8.564630e+10,4.460675e+10,2.340305e+10,2.176946e+10


In [52]:
liability_df.isna().sum()

KIS            0
Stock          0
Name           0
부채총계 (2017)    0
부채총계 (2018)    0
부채총계 (2019)    0
부채총계 (2020)    0
부채총계 (2021)    0
dtype: int64

In [54]:
# 결측치 처리

# 2017년 영업활동으로인한현금흐름 채우기
#'296640 1500808000 / 402030 143967000 / 399720 825562000 / 288980 100009000 / 251120 296741000 / 372800 346799000 / 382840 5415355000 / 367000 664806000 / 314930 1268019000 / 356890 777600000 / 352770 -690619000 / 337930 452401000 / 347770 1553852000'

piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='296640'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 1500808000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='402030'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 143967000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='399720'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 825562000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='288980'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 100009000

piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='251120'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 296741000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='372800'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 346799000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='382840'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 5415355000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='367000'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 664806000

piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='314930'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 1268019000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='356890'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 777600000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='352770'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = -690619000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='337930'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 452401000
piotroski_df.loc[list(piotroski_df[piotroski_df['Stock']=='347770'].index.values).pop(), '영업활동으로인한현금흐름 (2017)' ] = 1553852000



# # df_2['C2_New'] = np.where(pd.notnull(df_2['C2']) == True, df_2['C2'], df_2['C1'])

# ttmp['초기온도(℃)_x'] = np.where(pd.notnull(ttmp['초기온도(℃)_x'])==True, ttmp['초기온도(℃)_x'], ttmp['초기온도(℃)_y'])
# ttmp.isna().sum()



# 비유동부채 = 부채총계 - 유동부채
piotroski_df = pd.merge(piotroski_df, liability_df, how= 'left', on = 'Stock')
piotroski_df['비유동부채(계) (2017)'] =  np.where(pd.notnull(piotroski_df['비유동부채(계) (2017)'])==True, piotroski_df['비유동부채(계) (2017)'], piotroski_df['부채총계 (2017)'] - piotroski_df['유동부채(계) (2017)'] )
piotroski_df['비유동부채(계) (2018)'] =  np.where(pd.notnull(piotroski_df['비유동부채(계) (2018)'])==True, piotroski_df['비유동부채(계) (2018)'], piotroski_df['부채총계 (2018)'] - piotroski_df['유동부채(계) (2018)'] )
piotroski_df['비유동부채(계) (2019)'] =  np.where(pd.notnull(piotroski_df['비유동부채(계) (2019)'])==True, piotroski_df['비유동부채(계) (2019)'], piotroski_df['부채총계 (2019)'] - piotroski_df['유동부채(계) (2019)'] )
piotroski_df['비유동부채(계) (2020)'] =  np.where(pd.notnull(piotroski_df['비유동부채(계) (2020)'])==True, piotroski_df['비유동부채(계) (2020)'], piotroski_df['부채총계 (2020)'] - piotroski_df['유동부채(계) (2020)'] )
piotroski_df['비유동부채(계) (2021)'] =  np.where(pd.notnull(piotroski_df['비유동부채(계) (2021)'])==True, piotroski_df['비유동부채(계) (2021)'], piotroski_df['부채총계 (2021)'] - piotroski_df['유동부채(계) (2021)'] )

In [55]:
piotroski_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1279 entries, 0 to 1278
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   KIS_x                1279 non-null   object 
 1   Stock                1279 non-null   object 
 2   Name_x               1279 non-null   object 
 3   총자본회전율 (2017)        1279 non-null   float64
 4   총자본영업이익율 (2017)      1279 non-null   float64
 5   매출총이익(손실) (2017)     1279 non-null   float64
 6   당기순이익(손실) (2017)     1279 non-null   float64
 7   유동자산(계) (2017)       1279 non-null   float64
 8   유동부채(계) (2017)       1279 non-null   float64
 9   비유동부채(계) (2017)      1279 non-null   float64
 10  영업활동으로인한현금흐름 (2017)  1279 non-null   float64
 11  총자본회전율 (2018)        1279 non-null   float64
 12  총자본영업이익율 (2018)      1279 non-null   float64
 13  매출총이익(손실) (2018)     1279 non-null   float64
 14  당기순이익(손실) (2018)     1279 non-null   float64
 15  유동자산(계) (2018)       1279 non-null   f

In [56]:
piotroski_df

,KIS_x,Stock,Name_x,총자본회전율 (2017),총자본영업이익율 (2017),매출총이익(손실) (2017),당기순이익(손실) (2017),유동자산(계) (2017),유동부채(계) (2017),비유동부채(계) (2017),영업활동으로인한현금흐름 (2017),총자본회전율 (2018),총자본영업이익율 (2018),매출총이익(손실) (2018),당기순이익(손실) (2018),유동자산(계) (2018),유동부채(계) (2018),비유동부채(계) (2018),영업활동으로인한현금흐름 (2018),총자본회전율 (2019),총자본영업이익율 (2019),매출총이익(손실) (2019),당기순이익(손실) (2019),유동자산(계) (2019),유동부채(계) (2019),비유동부채(계) (2019),영업활동으로인한현금흐름 (2019),총자본회전율 (2020),총자본영업이익율 (2020),매출총이익(손실) (2020),당기순이익(손실) (2020),유동자산(계) (2020),유동부채(계) (2020),비유동부채(계) (2020),영업활동으로인한현금흐름 (2020),총자본회전율 (2021),총자본영업이익율 (2021),매출총이익(손실) (2021),당기순이익(손실) (2021),유동자산(계) (2021),유동부채(계) (2021),비유동부채(계) (2021),영업활동으로인한현금흐름 (2021),2017_유상증자여부,2018_유상증자여부,2019_유상증자여부,2020_유상증자여부,KIS_y,Name_y,부채총계 (2017),부채총계 (2018),부채총계 (2019),부채총계 (2020),부채총계 (2021)
0,036644,389500,(주)에스비비테크,0.53,4.37,2.248718e+09,2.678790e+08,7.090299e+09,7.257253e+09,1.411578e+09,7.497610e+08,0.44,-5.23,1.620293e+09,-1.917031e+09,1.356880e+10,8.723921e+09,1.099427e+10,6.075800e+07,0.27,-17.06,1.152950e+08,-4.789125e+09,8.293176e+09,3.591423e+09,1.568548e+10,-2.686112e+09,0.38,-15.18,1.080739e+09,-3.276516e+09,4.417464e+09,7.995986e+09,5.215853e+09,-1.165070e+09,0.42,-13.65,1.165755e+09,-2.815924e+09,7.302344e+09,6.321137e+09,4.714673e+09,-1.715607e+09,0,0,0,0,036644,(주)에스비비테크,8.668831e+09,1.971819e+10,1.927691e+10,1.321184e+10,1.103581e+10
1,497730,296640,(주)이노룰스,1.20,18.12,6.174259e+09,7.684080e+08,4.828215e+09,5.665559e+09,1.045236e+09,1.500808e+09,0.75,3.88,5.928031e+09,1.770280e+08,6.745808e+09,7.579176e+09,1.856800e+09,1.148444e+09,0.84,17.43,7.203013e+09,6.186920e+08,7.552133e+09,4.520995e+09,2.587218e+09,4.188603e+09,0.74,8.25,6.224520e+09,1.591312e+09,7.703371e+09,5.275242e+09,6.478850e+08,1.613872e+09,1.05,21.71,1.000607e+10,3.009484e+09,1.128946e+10,3.720986e+09,7.612350e+08,2.473444e+09,0,0,0,0,497730,(주)이노룰스,6.710795e+09,9.435975e+09,7.108213e+09,5.923127e+09,4.482222e+09
2,499459,417970,모델솔루션(주),1.49,15.95,1.145713e+10,3.048896e+09,8.925932e+09,6.779529e+09,1.290421e+10,3.615779e+09,1.49,25.30,1.561273e+10,7.210181e+09,1.865476e+10,8.760334e+09,1.269288e+10,5.469434e+09,1.40,21.58,1.661634e+10,7.270528e+09,2.073613e+10,1.861937e+10,2.074573e+09,1.075707e+10,1.26,15.68,1.370890e+10,1.299209e+09,2.207469e+10,1.759698e+10,1.989822e+09,8.304769e+09,1.33,18.82,1.607404e+10,7.086006e+09,2.716175e+10,9.062293e+09,1.216936e+10,9.397543e+09,0,0,0,0,499459,모델솔루션(주),1.968374e+10,2.145321e+10,2.069394e+10,1.958681e+10,2.123166e+10
3,HS8995,314140,(주)알피바이오,1.11,11.02,1.321373e+10,5.881569e+09,3.334482e+10,1.454381e+10,1.001158e+10,1.023374e+10,0.84,8.28,1.254839e+10,4.363446e+09,4.295203e+10,3.000506e+10,6.890256e+09,6.995499e+09,0.68,3.85,1.130269e+10,3.423335e+09,3.311131e+10,3.332302e+10,2.587282e+10,6.582410e+09,0.79,3.37,1.263887e+10,4.615509e+09,4.319151e+10,6.830001e+10,1.111733e+09,6.454520e+09,0.84,4.29,1.293782e+10,3.812561e+09,5.227193e+10,7.545094e+10,3.359074e+09,9.459383e+09,0,0,0,0,HS8995,(주)알피바이오,2.455539e+10,3.689531e+10,5.919584e+10,6.941174e+10,7.881001e+10
4,867055,129920,(주)대성하이텍,0.75,6.10,1.715736e+10,2.296441e+09,7.778543e+10,4.514462e+10,3.186916e+10,9.278596e+09,0.74,3.96,1.541051e+10,1.865670e+09,7.966123e+10,4.880179e+10,2.796324e+10,8.755060e+08,0.58,4.13,1.635153e+10,3.099913e+09,8.274118e+10,4.780276e+10,2.722766e+10,-5.228960e+09,0.59,-6.91,3.265241e+09,-1.173379e+10,6.825128e+10,5.116868e+10,3.553830e+10,1.280408e+10,0.79,8.96,2.304886e+10,7.457426e+09,8.730808e+10,8.915013e+10,2.263330e+10,8.944090e+08,0,0,0,0,867055,(주)대성하이텍,7.701378e+10,7.676503e+10,7.503042e+10,8.670698e+10,1.117834e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),0.70,12.53,1.511315e+10,2.833042e+09,1.303828e+10,6.999535e+09,4.228802e+09,4.281560e+09,0.45,8.05,1.663058e+10,3.963095e+09,1.317464e+10,2.285635e+10,1.800394e

# 유상증자 기업여부 확인

In [57]:
유상증자_2017 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2017_유상증자기업.csv')
유상증자_2018 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2018_유상증자기업.csv')
유상증자_2019 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2019_유상증자기업.csv')
유상증자_2020 = pd.read_csv('/content/drive/MyDrive/핀테크 파이널/data/2020_유상증자기업.csv')

In [58]:
유상증자_2017

,번호,시간,회사명,종목코드,공시제목,제출인
0,1025,2017-01-03 16:36,SBW생명과학,151910,[정정]유상증자결정(제3자배정),에스비더블유생명과학
1,1024,2017-01-03 16:36,SBW생명과학,151910,[정정]유상증자결정(제3자배정),에스비더블유생명과학
2,1023,2017-01-04 15:17,에이아이비트,39230,[정정]유상증자결정(제3자배정),제이앤케이인더스트리
3,1022,2017-01-04 17:17,인크로스,216050,유상증자결정(제3자배정),인크로스
4,1021,2017-01-05 15:45,비엔씨컴퍼니,58370,[정정]유상증자결정,비엔씨컴퍼니
...,...,...,...,...,...,...
1020,5,2017-12-29 16:14,한일진공,123840,[정정]유상증자결정(주주배정후 실권주 일반공모)[정],한일진공
1021,4,2017-12-29 16:46,WI,73570,[정정]유상증자결정(제3자배정),더블유아이
1022,3,2017-12-29 17:56,세영디앤씨,52190,[정정]유상증자결정(제3자배정)[정],세영디앤씨
1023,2,2017-12-29 17:58,에스에이치엔엘,50320,유상증자결정(제3자배정)[정],에스에이치엔엘


In [59]:
# 유상증자결정인 종목코드만 불러오기
def make_preprocessing_유상증자(df, piotroski_df, year):
  # 종목코드 6자리로 채워주기
  df['종목코드'] = df['종목코드'].astype('str')
  df['종목코드'] = df['종목코드'].apply(lambda x: x.zfill(6) if len(x)!=6 else x) # 종목코드 6자리로 만들어주기


  # 유상증자여부 - 유상증자면 1로 일단 라벨링
  df['유상증자여부'] = 0
  df['유상증자여부'] = df.apply(lambda x: 1 if '유상증자결정' in x['공시제목'] else 0, axis=1)
  
  유상증자결정기업_list = list(df[df['유상증자여부']==1]['종목코드'])
  
  piotroski_df[f'{year}_유상증자여부'] = piotroski_df.apply(lambda x: 1 if x['Stock'] in 유상증자결정기업_list else 0, axis=1)


  return piotroski_df


piotroski_df = make_preprocessing_유상증자(유상증자_2017, piotroski_df, 2017)
piotroski_df = make_preprocessing_유상증자(유상증자_2018, piotroski_df, 2018)
piotroski_df = make_preprocessing_유상증자(유상증자_2019, piotroski_df, 2019)
piotroski_df = make_preprocessing_유상증자(유상증자_2020, piotroski_df, 2020)

piotroski_df

,KIS_x,Stock,Name_x,총자본회전율 (2017),총자본영업이익율 (2017),매출총이익(손실) (2017),당기순이익(손실) (2017),유동자산(계) (2017),유동부채(계) (2017),비유동부채(계) (2017),영업활동으로인한현금흐름 (2017),총자본회전율 (2018),총자본영업이익율 (2018),매출총이익(손실) (2018),당기순이익(손실) (2018),유동자산(계) (2018),유동부채(계) (2018),비유동부채(계) (2018),영업활동으로인한현금흐름 (2018),총자본회전율 (2019),총자본영업이익율 (2019),매출총이익(손실) (2019),당기순이익(손실) (2019),유동자산(계) (2019),유동부채(계) (2019),비유동부채(계) (2019),영업활동으로인한현금흐름 (2019),총자본회전율 (2020),총자본영업이익율 (2020),매출총이익(손실) (2020),당기순이익(손실) (2020),유동자산(계) (2020),유동부채(계) (2020),비유동부채(계) (2020),영업활동으로인한현금흐름 (2020),총자본회전율 (2021),총자본영업이익율 (2021),매출총이익(손실) (2021),당기순이익(손실) (2021),유동자산(계) (2021),유동부채(계) (2021),비유동부채(계) (2021),영업활동으로인한현금흐름 (2021),2017_유상증자여부,2018_유상증자여부,2019_유상증자여부,2020_유상증자여부,KIS_y,Name_y,부채총계 (2017),부채총계 (2018),부채총계 (2019),부채총계 (2020),부채총계 (2021)
0,036644,389500,(주)에스비비테크,0.53,4.37,2.248718e+09,2.678790e+08,7.090299e+09,7.257253e+09,1.411578e+09,7.497610e+08,0.44,-5.23,1.620293e+09,-1.917031e+09,1.356880e+10,8.723921e+09,1.099427e+10,6.075800e+07,0.27,-17.06,1.152950e+08,-4.789125e+09,8.293176e+09,3.591423e+09,1.568548e+10,-2.686112e+09,0.38,-15.18,1.080739e+09,-3.276516e+09,4.417464e+09,7.995986e+09,5.215853e+09,-1.165070e+09,0.42,-13.65,1.165755e+09,-2.815924e+09,7.302344e+09,6.321137e+09,4.714673e+09,-1.715607e+09,0,0,0,0,036644,(주)에스비비테크,8.668831e+09,1.971819e+10,1.927691e+10,1.321184e+10,1.103581e+10
1,497730,296640,(주)이노룰스,1.20,18.12,6.174259e+09,7.684080e+08,4.828215e+09,5.665559e+09,1.045236e+09,1.500808e+09,0.75,3.88,5.928031e+09,1.770280e+08,6.745808e+09,7.579176e+09,1.856800e+09,1.148444e+09,0.84,17.43,7.203013e+09,6.186920e+08,7.552133e+09,4.520995e+09,2.587218e+09,4.188603e+09,0.74,8.25,6.224520e+09,1.591312e+09,7.703371e+09,5.275242e+09,6.478850e+08,1.613872e+09,1.05,21.71,1.000607e+10,3.009484e+09,1.128946e+10,3.720986e+09,7.612350e+08,2.473444e+09,0,0,0,0,497730,(주)이노룰스,6.710795e+09,9.435975e+09,7.108213e+09,5.923127e+09,4.482222e+09
2,499459,417970,모델솔루션(주),1.49,15.95,1.145713e+10,3.048896e+09,8.925932e+09,6.779529e+09,1.290421e+10,3.615779e+09,1.49,25.30,1.561273e+10,7.210181e+09,1.865476e+10,8.760334e+09,1.269288e+10,5.469434e+09,1.40,21.58,1.661634e+10,7.270528e+09,2.073613e+10,1.861937e+10,2.074573e+09,1.075707e+10,1.26,15.68,1.370890e+10,1.299209e+09,2.207469e+10,1.759698e+10,1.989822e+09,8.304769e+09,1.33,18.82,1.607404e+10,7.086006e+09,2.716175e+10,9.062293e+09,1.216936e+10,9.397543e+09,0,0,0,0,499459,모델솔루션(주),1.968374e+10,2.145321e+10,2.069394e+10,1.958681e+10,2.123166e+10
3,HS8995,314140,(주)알피바이오,1.11,11.02,1.321373e+10,5.881569e+09,3.334482e+10,1.454381e+10,1.001158e+10,1.023374e+10,0.84,8.28,1.254839e+10,4.363446e+09,4.295203e+10,3.000506e+10,6.890256e+09,6.995499e+09,0.68,3.85,1.130269e+10,3.423335e+09,3.311131e+10,3.332302e+10,2.587282e+10,6.582410e+09,0.79,3.37,1.263887e+10,4.615509e+09,4.319151e+10,6.830001e+10,1.111733e+09,6.454520e+09,0.84,4.29,1.293782e+10,3.812561e+09,5.227193e+10,7.545094e+10,3.359074e+09,9.459383e+09,0,0,0,0,HS8995,(주)알피바이오,2.455539e+10,3.689531e+10,5.919584e+10,6.941174e+10,7.881001e+10
4,867055,129920,(주)대성하이텍,0.75,6.10,1.715736e+10,2.296441e+09,7.778543e+10,4.514462e+10,3.186916e+10,9.278596e+09,0.74,3.96,1.541051e+10,1.865670e+09,7.966123e+10,4.880179e+10,2.796324e+10,8.755060e+08,0.58,4.13,1.635153e+10,3.099913e+09,8.274118e+10,4.780276e+10,2.722766e+10,-5.228960e+09,0.59,-6.91,3.265241e+09,-1.173379e+10,6.825128e+10,5.116868e+10,3.553830e+10,1.280408e+10,0.79,8.96,2.304886e+10,7.457426e+09,8.730808e+10,8.915013e+10,2.263330e+10,8.944090e+08,0,0,0,0,867055,(주)대성하이텍,7.701378e+10,7.676503e+10,7.503042e+10,8.670698e+10,1.117834e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,I73639,950110,SBI핀테크솔루션즈(주),0.70,12.53,1.511315e+10,2.833042e+09,1.303828e+10,6.999535e+09,4.228802e+09,4.281560e+09,0.45,8.05,1.663058e+10,3.963095e+09,1.317464e+10,2.285635e+10,1.800394e

In [60]:
# 이거는 전년도 기준으로 보기때문에 2021년도 데이터가 없고 2017년 데이터가 있다.
print(piotroski_df['2017_유상증자여부'].value_counts())
print(piotroski_df['2018_유상증자여부'].value_counts())
print(piotroski_df['2019_유상증자여부'].value_counts())
print(piotroski_df['2020_유상증자여부'].value_counts())

0    1143
1     136
Name: 2017_유상증자여부, dtype: int64
0    1123
1     156
Name: 2018_유상증자여부, dtype: int64
0    1151
1     128
Name: 2019_유상증자여부, dtype: int64
0    1141
1     138
Name: 2020_유상증자여부, dtype: int64


In [79]:
def piotroski_f_score(df):
  year = ['2018', '2019', '2020', '2021']
  for y in year:

    for col in df.columns:
      if y in col:
        if '당기순이익(손실)' in col:
          당기순이익 = col
        if '총자본영업이익율' in col:
          총자본영업이익율 = col
        if '영업활동으로인한현금흐름' in col:
          영업활동으로인한현금흐름 = col

        if f'유동부채(계) ({y})' == col:
          유동부채 = col
        if f'유동자산(계) ({y})' in col:
          유동자산 = col

        if '비유동부채' in col:
          비유동부채 = col
        if '매출총이익' in col:
          매출총이익 = col
        if '총자본회전율' in col:
          총자본회전율 = col


      elif str(int(y)-1) in col: # 전년도 데이터 얻어야하는 것만
        if '비유동부채' in col:
          전년비유동부채 = col
        if f'유동자산(계) ({str(int(y)-1)})' == col:
          전년유동자산 = col
        if f'유동부채(계) ({str(int(y)-1)})' == col:
          전년유동부채 = col
        if '매출총이익' in col:
          전년매출총이익 = col
        if '총자본회전율' in col:
          전년총자본회전율 = col
        if '유상증자여부' in col:
          전년유상증자 = col
   

    # 1) 당기순이익 > 0이면 1부여
    df[f'{y}_x1'] = df.apply(lambda x: 1 if x[f'{당기순이익}']>0 else 0,axis=1)  
    
    # 2) 총자본영업이익률 > 0 이면 1부여
    df[f'{y}_x2'] = df.apply(lambda x: 1 if x[f'{총자본영업이익율}']>0 else 0, axis=1)

    # 3) 영업활동으로 인한 현금흐름 >0이면 1 부여
    df[f'{y}_x3'] = df.apply(lambda x:1 if x[f'{영업활동으로인한현금흐름}'] >0 else 0, axis=1)

    # 4) 영업활동으로 인한 현금흐름 > 당기순이익이면 1 부여
    df[f'{y}_x4'] = df.apply(lambda x:1 if x[f'{영업활동으로인한현금흐름}'] > x[f'{당기순이익}'] else 0, axis=1)

    # 5) 전년도 비유동부채 > 올해 비유동부채
    df[f'{y}_x5'] = df.apply(lambda x:1 if x[f'{전년비유동부채}'] > x[f'{비유동부채}'] else 0, axis=1)

    # 6) 전년유동자산 / 전년유동부채 < 올해유동자산 / 올해유동부채
    
    df[f'{y}_x6'] = df.apply(lambda x:1 if (x[f'{전년유동자산}'] / x[f'{전년유동부채}']) < (x[f'{유동자산}'] / x[f'{유동부채}']) else 0, axis=1)

    # 7) 전년에 유상증자여부가 1이면 0, 유상증자 아니면 좋은거니까 1
    df[f'{y}_x7'] = df.apply(lambda x:0 if x[f'{전년유상증자}']==1 else 1, axis=1)
    
    # 8) 전년매출총이익 < 올해매출총이익이면 1, 아니면 0
    df[f'{y}_x8'] = df.apply(lambda x:1 if x[f'{매출총이익}'] > x[f'{전년매출총이익}'] else 0, axis=1)

    # 9) 전년총자본회전율 < 올해총자본회전율
    df[f'{y}_x9'] = df.apply(lambda x:1 if x[f'{총자본회전율}'] > x[f'{전년총자본회전율}'] else 0, axis=1)

    df[f'{y}_F_score'] =  df[f'{y}_x1'] + df[f'{y}_x2'] + df[f'{y}_x3'] + df[f'{y}_x4'] + df[f'{y}_x5'] + df[f'{y}_x6'] + df[f'{y}_x7'] + df[f'{y}_x8'] + df[f'{y}_x9']
  return df

In [80]:
tt = piotroski_f_score(piotroski_df)
tt

,KIS_x,Stock,Name_x,총자본회전율 (2017),총자본영업이익율 (2017),매출총이익(손실) (2017),당기순이익(손실) (2017),유동자산(계) (2017),유동부채(계) (2017),비유동부채(계) (2017),영업활동으로인한현금흐름 (2017),총자본회전율 (2018),총자본영업이익율 (2018),매출총이익(손실) (2018),당기순이익(손실) (2018),유동자산(계) (2018),유동부채(계) (2018),비유동부채(계) (2018),영업활동으로인한현금흐름 (2018),총자본회전율 (2019),총자본영업이익율 (2019),매출총이익(손실) (2019),당기순이익(손실) (2019),유동자산(계) (2019),유동부채(계) (2019),비유동부채(계) (2019),영업활동으로인한현금흐름 (2019),총자본회전율 (2020),총자본영업이익율 (2020),매출총이익(손실) (2020),당기순이익(손실) (2020),유동자산(계) (2020),유동부채(계) (2020),비유동부채(계) (2020),영업활동으로인한현금흐름 (2020),총자본회전율 (2021),총자본영업이익율 (2021),매출총이익(손실) (2021),당기순이익(손실) (2021),유동자산(계) (2021),유동부채(계) (2021),비유동부채(계) (2021),영업활동으로인한현금흐름 (2021),2017_유상증자여부,2018_유상증자여부,2019_유상증자여부,2020_유상증자여부,KIS_y,Name_y,부채총계 (2017),부채총계 (2018),부채총계 (2019),부채총계 (2020),부채총계 (2021),2018_F_score,2019_F_score,2020_F_score,2021_F_score,2018_x1,2019_x1,2020_x1,2021_x1,2018_x2,2018_x3,2018_x4,2018_x5,2018_x6,2018_x7,2018_x8,2018_x9,2019_x2,2019_x3,2019_x4,2019_x5,2019_x6,2019_x7,2019_x8,2019_x9,2020_x2,2020_x3,2020_x4,2020_x5,2020_x6,2020_x7,2020_x8,2020_x9,2021_x2,2021_x3,2021_x4,2021_x5,2021_x6,2021_x7,2021_x8,2021_x9
0,036644,389500,(주)에스비비테크,0.53,4.37,2.248718e+09,2.678790e+08,7.090299e+09,7.257253e+09,1.411578e+09,7.497610e+08,0.44,-5.23,1.620293e+09,-1.917031e+09,1.356880e+10,8.723921e+09,1.099427e+10,6.075800e+07,0.27,-17.06,1.152950e+08,-4.789125e+09,8.293176e+09,3.591423e+09,1.568548e+10,-2.686112e+09,0.38,-15.18,1.080739e+09,-3.276516e+09,4.417464e+09,7.995986e+09,5.215853e+09,-1.165070e+09,0.42,-13.65,1.165755e+09,-2.815924e+09,7.302344e+09,6.321137e+09,4.714673e+09,-1.715607e+09,0,0,0,0,036644,(주)에스비비테크,8.668831e+09,1.971819e+10,1.927691e+10,1.321184e+10,1.103581e+10,4,3,5,6,0,0,0,0,0,1,1,0,1,1,0,0,0,0,1,0,1,1,0,0,0,0,1,1,0,1,1,1,0,0,1,1,1,1,1,1
1,497730,296640,(주)이노룰스,1.20,18.12,6.174259e+09,7.684080e+08,4.828215e+09,5.665559e+09,1.045236e+09,1.500808e+09,0.75,3.88,5.928031e+09,1.770280e+08,6.745808e+09,7.579176e+09,1.856800e+09,1.148444e+09,0.84,17.43,7.203013e+09,6.186920e+08,7.552133e+09,4.520995e+09,2.587218e+09,4.188603e+09,0.74,8.25,6.224520e+09,1.591312e+09,7.703371e+09,5.275242e+09,6.478850e+08,1.613872e+09,1.05,21.71,1.000607e+10,3.009484e+09,1.128946e+10,3.720986e+09,7.612350e+08,2.473444e+09,0,0,0,0,497730,(주)이노룰스,6.710795e+09,9.435975e+09,7.108213e+09,5.923127e+09,4.482222e+09,6,8,6,7,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,0,1,0,0,1,1,0,0,1,1,1,1
2,499459,417970,모델솔루션(주),1.49,15.95,1.145713e+10,3.048896e+09,8.925932e+09,6.779529e+09,1.290421e+10,3.615779e+09,1.49,25.30,1.561273e+10,7.210181e+09,1.865476e+10,8.760334e+09,1.269288e+10,5.469434e+09,1.40,21.58,1.661634e+10,7.270528e+09,2.073613e+10,1.861937e+10,2.074573e+09,1.075707e+10,1.26,15.68,1.370890e+10,1.299209e+09,2.207469e+10,1.759698e+10,1.989822e+09,8.304769e+09,1.33,18.82,1.607404e+10,7.086006e+09,2.716175e+10,9.062293e+09,1.216936e+10,9.397543e+09,0,0,0,0,499459,모델솔루션(주),1.968374e+10,2.145321e+10,2.069394e+10,1.958681e+10,2.123166e+10,7,7,7,8,1,1,1,1,1,1,0,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,1,1,1,0,1,1,1,1
3,HS8995,314140,(주)알피바이오,1.11,11.02,1.321373e+10,5.881569e+09,3.334482e+10,1.454381e+10,1.001158e+10,1.023374e+10,0.84,8.28,1.254839e+10,4.363446e+09,4.295203e+10,3.000506e+10,6.890256e+09,6.995499e+09,0.68,3.85,1.130269e+10,3.423335e+09,3.311131e+10,3.332302e+10,2.587282e+10,6.582410e+09,0.79,3.37,1.263887e+10,4.615509e+09,4.319151e+10,6.830001e+10,1.111733e+09,6.454520e+09,0.84,4.29,1.293782e+10,3.812561e+09,5.227193e+10,7.545094e+10,3.359074e+09,9.459383e+09,0,0,0,0,HS8995,(주)알피바이오,2.455539e+10,3.689531e+10,5.919584e+10,6.941174e+10,7.881001e+10,6,5,8,8,1,1,1,1,1,1,1,1,0,1,0,0,1,1,1,0,0,1,0,0,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1
4,867055,129920,(주)대성하이텍,0.75,6.10,1.715736e+10,2.296441e+09,7.778543e+10,4.514462e+10,3.186916e+10,9.278596e+09,0.74,3.96,1.541051e+10,1.865670e+09,7.966123e+10,4.880179e+10,2.796324e+10,8.755060e+08,0.58,4.13,1.635153e+10,3.099913e+09,8.274118e+10,4.780276e+10,2.722766e+10,-5.228960e+09,0.59,-6.91,3.265241e

In [81]:
print(piotroski_df['2018_F_score'].value_counts())
print(piotroski_df['2019_F_score'].value_counts())
print(piotroski_df['2020_F_score'].value_counts())
print(piotroski_df['2021_F_score'].value_counts())

6    290
7    256
5    246
4    134
8    132
3    115
2     65
9     26
1     13
0      2
Name: 2018_F_score, dtype: int64
6    295
7    271
5    212
4    150
8    136
3    117
2     55
9     26
1     16
0      1
Name: 2019_F_score, dtype: int64
6    267
7    261
5    198
4    159
8    158
3    113
2     66
9     43
1     13
0      1
Name: 2020_F_score, dtype: int64
6    280
7    257
5    188
8    181
4    138
3    107
2     59
9     52
1     17
Name: 2021_F_score, dtype: int64


In [83]:
final_piotroski = piotroski_df[['Stock', '2018_F_score', '2019_F_score', '2020_F_score', '2021_F_score']]
final_piotroski.to_csv('piotroski.csv',index=False)

In [84]:
final_piotroski

,Stock,2018_F_score,2019_F_score,2020_F_score,2021_F_score
0,389500,4,3,5,6
1,296640,6,8,6,7
2,417970,7,7,7,8
3,314140,6,5,8,8
4,129920,5,6,4,7
...,...,...,...,...,...
1274,950110,4,5,7,4
1275,046970,7,5,6,6
1276,113810,5,7,5,6
1277,155650,7,2,8,7
